In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
import panel as pn
import datetime 

In [ ]:
# Carrega as variáveis do arquivo .env
load_dotenv()

In [ ]:
# Lê as variáveis de ambiente
DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

In [ ]:
# Função para conectar ao banco de dados
def get_connection():
    conn = pg.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASS
    )
    return conn

In [ ]:
#funções auxiliares
def get_profissionais_por_tipo(tipo):
    conn = get_connection()
    df = pd.read_sql("SELECT p.id, pessoa.pnome || ' ' || COALESCE(pessoa.mnome, '') || ' ' || pessoa.unome AS nome FROM profissional p JOIN pessoa ON p.id = pessoa.id WHERE LOWER(p.cargo) = LOWER(%s)", conn, params=(tipo,))
    conn.close()
    return {row['nome']: row['id'] for _, row in df.iterrows()}


def get_criancas():
    conn = get_connection()
    df = pd.read_sql("SELECT c.id, pessoa.pnome || ' ' || COALESCE(pessoa.mnome, '') || ' ' || pessoa.unome AS nome FROM crianca_adolescente c JOIN pessoa ON c.id = pessoa.id", conn)
    conn.close()
    return {row['nome']: row['id'] for _, row in df.iterrows()}



In [ ]:
# Campo Select de Profissional que será atualizado
id_profissional = pn.widgets.Select(name="Profissional", options={})


In [ ]:
# Função para agendar atendimento
def agendar_atendimento(data, hora, status, frequencia, tipo, id_profissional, id_crianca_adolescente):
    conn = get_connection()
    cursor = conn.cursor()
    
    try:
        # Inserir dados de atendimento
        cursor.execute('''
            INSERT INTO Agenda_Atendimento(data, hora, status, frequencia, tipo, id_profissional, id_crianca_adolescente)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        ''', (data, hora, status, frequencia, tipo, id_profissional, id_crianca_adolescente))
        
        conn.commit()
        return "Atendimento agendado com sucesso!"
    
    except Exception as e:
        return f"Erro ao agendar atendimento: {e}"
    
    finally:
        cursor.close()
        conn.close()

In [ ]:
# Widgets para inserir dados
data = pn.widgets.DatePicker(name="Data do Atendimento")
hora = pn.widgets.TimePicker(name="Hora do Atendimento", value=datetime.time(10, 0))
status = pn.widgets.Select(
    name="Status de Atendimento", 
    options=["Agendado", "Concluido", "Cancelado"],
    value="Agendado"  # Definindo um valor padrão
)
frequencia = pn.widgets.Select(
    name="Frequência", 
    options=["Semanal", "Mensal", "Quizenal", "Bimestral"],
    value="Semanal"  # Definindo um valor padrão
)
# Alteração para usar o submódulo correto
tipo = pn.widgets.Select(
    name="Tipo de Atendimento", 
    options=["Psicólogo", "Pedagogo", "Assistente Social"],
    value="Psicólogo"  # Definindo um valor padrão
)

id_profissional = pn.widgets.Select(
    name="Profissional",
    options=get_profissionais_por_tipo(tipo.value)
)

id_crianca_adolescente = pn.widgets.Select(
    name="Criança/Adolescente", 
    options=get_criancas()
)

In [ ]:
def atualizar_profissionais(event):
    novos = get_profissionais_por_tipo(tipo.value)
    if novos:
        id_profissional.options = novos
        id_profissional.value = list(novos.values())[0]  # Seleciona o primeiro por padrão
    else:
        id_profissional.options = {}
        id_profissional.value = None


In [ ]:
tipo.param.watch(atualizar_profissionais, 'value')

In [ ]:
# Inicializa as extensões do Panel necessárias para exibir tabelas 
# interativas (Tabulator) e notificações na interface gráfica

pn.extension()
pn.extension('tabulator')
pn.extension(notifications=True)

In [ ]:
# Widget para inserir o ID do agendamento que será excluído
data_filtro = pn.widgets.DatePicker(name="Filtrar por Data")
hora_filtro = pn.widgets.TimePicker(name="Filtrar por Hora", value=None)

In [ ]:
# Texto explicativo para os filtros
texto_filtros = pn.pane.Markdown("""
**Filtros de Consulta:**
- Preencha a data para filtrar por um dia específico.
- Preencha a hora para filtrar por um horário específico (opcional).
- Se não preencher a hora, todos os horários do dia selecionado serão exibidos.
""")

In [ ]:
# Botão para limpar filtros
def limpar_filtros(event):
    data_filtro.value = None
    hora_filtro.value = None

button_limpar_filtros = pn.widgets.Button(name="Limpar Filtros", button_type="warning")
button_limpar_filtros.on_click(limpar_filtros)

In [ ]:
# Widget para inserir o ID do agendamento que será excluído
id_agendamento = pn.widgets.TextInput(name="ID do Agendamento para Excluir e Editar")  # Exemplo de valor padrão

In [ ]:
def consultar_agenda():
    """
    Consulta todos os registros na tabela 'Agenda_Atendimento' com nomes legíveis, com opção de filtro por data e hora.
    """
    try:
        conn = get_connection()
        
        query = """
            SELECT a.id, a.data, a.hora, a.status, a.frequencia, a.tipo, 
                   pessoa_prof.pnome || ' ' || COALESCE(pessoa_prof.mnome, '') || ' ' || pessoa_prof.unome AS profissional, 
                   pessoa_crianca.pnome || ' ' || COALESCE(pessoa_crianca.mnome, '') || ' ' || pessoa_crianca.unome AS crianca 
            FROM Agenda_Atendimento a 
            JOIN profissional p ON a.id_profissional = p.id 
            JOIN pessoa pessoa_prof ON p.id = pessoa_prof.id 
            JOIN crianca_adolescente c ON a.id_crianca_adolescente = c.id 
            JOIN pessoa pessoa_crianca ON c.id = pessoa_crianca.id 
        """
        params = []
        filtros = []
        if data_filtro.value:
            filtros.append("a.data = %s")
            params.append(data_filtro.value)
        if hora_filtro.value:
            filtros.append("a.hora = %s")
            params.append(str(hora_filtro.value))
        if filtros:
            query += " WHERE " + " AND ".join(filtros)
        query += " ORDER BY a.data, a.hora"
        
        df = pd.read_sql(query, conn, params=params)
        conn.close()

        if df.empty:
            return pn.pane.Alert("Nenhum agendamento encontrado!")

        return pn.pane.DataFrame(df, width=900, height=400)

    except Exception as e:
        return pn.pane.Alert(f"Erro na consulta: {e}")


In [ ]:
def on_excluir(event):
    """
    Função para excluir um agendamento.
    """
    try:
        # Conectar ao banco de dados
        conn = get_connection()
        cursor = conn.cursor()
        
        # Excluir o agendamento com base no ID fornecido
        cursor.execute("DELETE FROM Agenda_Atendimento WHERE id = %s", (id_agendamento.value,))
        
        conn.commit()
        cursor.close()
        conn.close()
        
        return pn.pane.Alert('Agendamento excluído com sucesso!')
    except Exception as e:
        return pn.pane.Alert(f'Erro ao excluir agendamento: {e}')


In [ ]:
def on_atualizar(event):
    if not id_agendamento.value:
        message.object = "Por favor, selecione um agendamento para atualizar."
        return

    try:
        # Conectar ao banco de dados
        conn = get_connection()
        cursor = conn.cursor()
        
        # Atualizar os dados do agendamento com base no ID fornecido (usando o widget id_agendamento)
        cursor.execute("UPDATE Agenda_Atendimento SET data = %s, hora = %s, status = %s, frequencia = %s, tipo = %s, id_profissional = %s, id_crianca_adolescente = %s WHERE id = %s", (
            data.value, 
            hora.value, 
            status.value, 
            frequencia.value, 
            tipo.value, 
            id_profissional.value, 
            id_crianca_adolescente.value, 
            id_agendamento.value  # O ID do agendamento que será atualizado
        ))
        
        # Confirmar a atualização
        conn.commit()
        cursor.close()
        conn.close()
        
        return pn.pane.Alert('Agendamento atualizado com sucesso!')
    except Exception as e:
        return pn.pane.Alert(f'Erro ao atualizar agendamento: {e}')


In [ ]:
# Função do botão para chamar a consulta
def on_consultar_click(event):
    """
    Chama a função de consulta e exibe a tabela no painel.
    """
    table = consultar_agenda()  # Chama a função que retorna a tabela de agendamentos
    if isinstance(table, pn.pane.DataFrame):
        message.object = ""  # Limpar a mensagem anterior
    else:
        message.object = table  # Exibir erro, caso haja
    if isinstance(table, pn.pane.DataFrame):
        table_panel.object = table.object  # Isso atualiza o DataFrame interno
        message.object = ""
    else:
        message.object = table

In [ ]:
# Botão para salvar
button = pn.widgets.Button(name="Agendar Atendimento", button_type="primary")
# Botão para realizar a consulta (mostrar todos os agendamentos)
button_consultar = pn.widgets.Button(name="Consultar Agenda", button_type="primary")
# botao para excluir
button_excluir = pn.widgets.Button(name="Excluir Agendamento", button_type="danger")
# Conectar o botão à função
button_atualizar = pn.widgets.Button(name="Atualizar Agendamento", button_type="primary")

In [ ]:
# Função do botão
def on_agendar_click(event):
    result = agendar_atendimento(
        data.value,
        hora.value,
        status.value,
        frequencia.value,
        tipo.value,
        id_profissional.value,
        id_crianca_adolescente.value
    )
    message.object = result

In [ ]:
# Mensagem de confirmação
message = pn.pane.Markdown("")
table_panel = pn.pane.DataFrame(pd.DataFrame(), width=1000, height=400)

In [ ]:
# Conectar o botão à função
button.on_click(on_agendar_click)
# Conectar o botão à função de consulta
button_consultar.on_click(on_consultar_click)
# Conectar o botão de exclusão à função de exclusão
button_excluir.on_click(on_excluir)
# Conectar o botão à função de atualizar
button_atualizar.on_click(on_atualizar)

In [ ]:
# Layout do painel
formulario = pn.Column(
    "### Agendamento de Atendimento",
    data,
    hora,
    status,
    frequencia,
    tipo,
    id_profissional,
    id_crianca_adolescente,
    texto_filtros,
    data_filtro,
    hora_filtro,
    button_limpar_filtros,
    id_agendamento,
    button,
    button_consultar,
    button_excluir,
    button_atualizar,
    message
)

In [ ]:
layout = pn.Row(
    formulario,
    pn.Spacer(width=50),  # espaço entre colunas (opcional)
    table_panel
)

In [ ]:
# Exibir painel
layout.servable()

In [ ]:
def preencher_campos_agendamento(event):
    agendamento_id = id_agendamento.value
    if not agendamento_id:
        return
    try:
        conn = get_connection()
        cursor = conn.cursor()
        cursor.execute("""
            SELECT data, hora, status, frequencia, tipo, id_profissional, id_crianca_adolescente
            FROM Agenda_Atendimento
            WHERE id = %s
        """, (agendamento_id,))
        result = cursor.fetchone()
        cursor.close()
        conn.close()
        if result:
            data.value = result[0]
            hora.value = result[1]
            status.value = result[2]
            frequencia.value = result[3]
            tipo.value = result[4]
            # Atualiza profissionais conforme o tipo
            atualizar_profissionais(None)
            id_profissional.value = result[5]
            id_crianca_adolescente.value = result[6]
        else:
            message.object = f"Agendamento com ID {agendamento_id} não encontrado."
    except Exception as e:
        message.object = f"Erro ao buscar agendamento: {e}"

id_agendamento.param.watch(preencher_campos_agendamento, 'value')